## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
!ollama pull deepseek-r1:8b

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [1]:
# imports

import requests
import ollama
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [10]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "deepseek-r1:8b"
MODEL = "deepseek-r1:1.5b"

In [3]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [5]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [6]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

In [8]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [12]:
display_summary("https://edwarddonner.com")

<think>
Alright, I need to create a summary of the website titled "Home - Edward Donner." Let's see what information is there.

First, it seems to be a platform related to LLMs, which stands for Large Language Models. There are mentions like "Connect Four," "Outsmart," and an arena for talent. The site also talks about Ed being the co-founder of Nebula.io, applying AI in discovering potential.

There are posts mentioning events like the Agentic AI Engineering Course on April 21, 2025, and a Welcome post on November 13, 2024. It's got some links to resources for mastering LLM engineering and an newsletter subscription.

The site has a navigation menu with options Connect Four, Outsmart, and Explore, all linked to LLMs. The About section is quite brief but mentions Ed as the co-founder of Nebula.io experimenting with LLMs.

There are several social media handles: LinkedIn, Twitter, Facebook, and a newsletter subscription link.

In summary, Edward Donner hosts a platform focused on AI and LLMs, promoting events and resources for discovering potential and managing talent. The site is navigated through Connect Four, Outsmart, or Explore sections.
</think>

Edward Donner operates a platform centered around Artificial Intelligence (AI) and Large Language Models (LLMs). It features events such as the Agentic AI Engineering Course on April 21, 2025, and an introductory Welcome post. The site highlights Ed's role as co-founder of Nebula.io, applying AI to discover talent. Social media handles include LinkedIn, Twitter, Facebook, and newsletter subscriptions.

Navigation:
- **Connect Four**: A game where players compete for a spot.
- **Outsmart**: An arena where LLMs face off in strategic challenges.
- **Explore**: The main page offering resources and events.

The platform promotes discovery of potential and talent management through various sections.